In [2]:
!pip install pandas
import os
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

###Lets import the dataset

In [3]:
df = pd.read_csv("/content/covid_19_daily_reports_us.csv")

###Let's take a look at the few records in the data:

In [4]:
df.head(10)

,Unnamed: 0,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,...,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
0,0,Alabama,US,2022-01-28 04:31:25,32.3182,-86.9023,1153149,16826,NaN,NaN,...,23518.366123,6901698.0,NaN,1.459135,84000001.0,USA,140759.485926,NaN,NaN,NaN
1,1,Alaska,US,2022-01-28 04:31:25,61.3707,-152.4044,206504,1081,NaN,NaN,...,28228.475350,3741734.0,NaN,0.523477,84000002.0,USA,511483.777485,NaN,NaN,NaN
2,2,American Samoa,US,2022-01-28 04:31:25,-14.2710,-170.1320,18,0,NaN,NaN,...,32.350245,2140.0,NaN,0.000000,16.0,ASM,3846.084722,NaN,NaN,NaN
3,3,Arizona,US,2022-01-28 04:31:25,33.7298,-111.4312,1813796,25932,NaN,NaN,...,24919.171882,17176458.0,NaN,1.429709,84000004.0,USA,235981.945719,NaN,NaN,NaN
4,4,Arkansas,US,2022-01-28 04:31:25,34.9697,-92.3731,762401,9596,NaN,NaN,...,25263.436592,4780951.0,NaN,1.258655,84000005.0,USA,158424.834747,NaN,NaN,NaN
5,5,California,US,2022-01-28 07:31:32,36.1162,-119.6816,8215130,79395,NaN,NaN,...,20792.203972,134288642.0,NaN,0.966470,84000006.0,USA,339866.076378,NaN,NaN,NaN
6,6,Colorado,US,2022-01-28 04:31:25,39.0598,-105.3111,1233278,11005,NaN,NaN,...,21415.775962,15700969.0,NaN,0.892337,84000008.0,USA,272646.098033,NaN,NaN,NaN
7,7,Connecticut,US,2022-01-28 04:31:25,41.5978,-72.7554,693386,9908,NaN,NaN,...,19448.251992,12710286.0,NaN,1.428930,84000009.0,USA,356501.061485,NaN,NaN,NaN
8,8,Delaware,US,2022-01-28 04:31:25,39.3185,-75.5071,244730,2494,NaN,NaN,...,25132.372936,2973672.0,NaN,1.019082,84000010.0,USA,305379.126770,NaN,NaN,NaN
9,9,Diamond Princess,US,2022-01-28 04:31:25,NaN,NaN,49,0,NaN,NaN,...,NaN,NaN,NaN,0.000000,84088888.0,USA,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41372 entries, 0 to 41371
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            41372 non-null  int64  
 1   Province_State        41372 non-null  object 
 2   Country_Region        41372 non-null  object 
 3   Last_Update           41353 non-null  object 
 4   Lat                   39928 non-null  float64
 5   Long_                 39928 non-null  float64
 6   Confirmed             41372 non-null  int64  
 7   Deaths                41372 non-null  int64  
 8   Recovered             15122 non-null  float64
 9   Active                15122 non-null  float64
 10  FIPS                  41353 non-null  float64
 11  Incident_Rate         39928 non-null  float64
 12  Total_Test_Results    27966 non-null  float64
 13  People_Hospitalized   5129 non-null   float64
 14  Case_Fatality_Ratio   28785 non-null  float64
 15  UID                

In [6]:
df['Last_Update']= pd.to_datetime(df['Last_Update'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41372 entries, 0 to 41371
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            41372 non-null  int64         
 1   Province_State        41372 non-null  object        
 2   Country_Region        41372 non-null  object        
 3   Last_Update           41353 non-null  datetime64[ns]
 4   Lat                   39928 non-null  float64       
 5   Long_                 39928 non-null  float64       
 6   Confirmed             41372 non-null  int64         
 7   Deaths                41372 non-null  int64         
 8   Recovered             15122 non-null  float64       
 9   Active                15122 non-null  float64       
 10  FIPS                  41353 non-null  float64       
 11  Incident_Rate         39928 non-null  float64       
 12  Total_Test_Results    27966 non-null  float64       
 13  People_Hospitali

In [7]:
df.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
Unnamed: 0,41372.0,20685.5,0.0,10342.75,20685.5,31028.25,41371.0,11943.212005
Last_Update,41353,2021-04-04 04:48:27.613885184,2020-04-12 23:18:15,2020-10-08 04:30:35,2021-04-04 04:30:32,2021-09-29 04:31:45,2022-03-26 04:31:08,NaN
Lat,39928.0,36.840089,-14.271,34.5946,39.06185,42.36165,61.3707,10.789527
Long_,39928.0,-85.206614,-170.1322,-101.165775,-87.9442,-76.970625,145.6739,49.308829
Confirmed,41372.0,512260.56224,0.0,33248.25,182856.5,624795.5,9088902.0,885843.844084
Deaths,41372.0,8400.130765,0.0,618.0,2981.5,10216.25,88579.0,13216.530073
Recovered,15122.0,85789.083984,0.0,3872.0,17548.5,84255.75,2470308.0,193330.31296
Active,15122.0,59369.15527,0.0,3166.0,14752.5,61115.0,1408516.0,119012.893004
FIPS,41353.0,3283.641791,1.0,18.0,32.0,48.0,99999.0,17246.352821
Incident_Rate,39928.0,8512.880588,0.0,1767.557282,8019.941656,12581.14579,33998.986181,7292.867748


In [8]:
df = df.sort_values(by=["Province_State", "Last_Update"])

In [9]:
df.head(20)

,Unnamed: 0,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,...,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
18913,18913,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3667,93,NaN,NaN,...,74.788122,NaN,437.0,NaN,84000001.0,USA,1265.075660,12.264945,62029.0,2.536133
23035,23035,Alabama,US,2020-04-13 23:07:54,32.3182,-86.9023,3870,99,NaN,NaN,...,78.928288,NaN,457.0,NaN,84000001.0,USA,1318.245181,12.238886,64636.0,2.558140
19901,19901,Alabama,US,2020-04-14 23:33:31,32.3182,-86.9023,4041,114,NaN,NaN,...,82.415817,NaN,493.0,NaN,84000001.0,USA,1377.818704,12.471541,67557.0,2.821084
36904,36904,Alabama,US,2020-04-15 22:56:51,32.3182,-86.9023,4307,118,NaN,NaN,...,87.840863,NaN,525.0,NaN,84000001.0,USA,1444.428468,12.883436,70823.0,2.739726
27038,27038,Alabama,US,2020-04-16 23:30:51,32.3182,-86.9023,4465,133,NaN,NaN,...,91.063258,NaN,553.0,NaN,84000001.0,USA,1506.734908,12.727273,73878.0,2.978723
19494,19494,Alabama,US,2020-04-17 23:30:52,32.3182,-86.9023,4557,148,NaN,NaN,...,92.939589,NaN,594.0,NaN,84000001.0,USA,1684.802837,12.994968,82609.0,3.247751
22338,22338,Alabama,US,2020-04-18 22:32:47,32.3182,-86.9023,4788,153,NaN,NaN,...,97.650813,NaN,620.0,NaN,84000001.0,USA,1748.924424,13.157895,85753.0,3.195489
33710,33710,Alabama,US,2020-04-19 23:41:01,32.3182,-86.9023,4978,157,NaN,NaN,...,101.525845,NaN,641.0,NaN,84000001.0,USA,1768.462744,13.113748,86711.0,3.153877
34117,34117,Alabama,US,2020-04-20 23:36:47,32.3182,-86.9023,5163,163,NaN,NaN,...,105.298903,NaN,641.0,NaN,84000001.0,USA,1817.328940,12.620595,89107.0,3.157079
2436,2436,Alabama,US,2020-04-21 23:40:34,32.3182,-86.9023,5404,183,NaN,NaN,...,110.214075,NaN,699.0,NaN,84000001.0,USA,1872.843876,13.146511,91829.0,3.386380


In [10]:
mask = (df['Province_State'] != 'Recovered') & (df['Province_State'] != 'American Samoa') & (df['Province_State'] != 'Diamond Princess') & (df['Province_State'] != 'Grand Princess') & (df['Province_State'] != 'Virgin Islands') & (df['Province_State'] != 'Northern Mariana Islands')
df = df[mask]


In [11]:
df

,Unnamed: 0,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,...,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
18913,18913,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3667,93,NaN,NaN,...,74.788122,NaN,437.0,NaN,84000001.0,USA,1265.075660,12.264945,62029.0,2.536133
23035,23035,Alabama,US,2020-04-13 23:07:54,32.3182,-86.9023,3870,99,NaN,NaN,...,78.928288,NaN,457.0,NaN,84000001.0,USA,1318.245181,12.238886,64636.0,2.558140
19901,19901,Alabama,US,2020-04-14 23:33:31,32.3182,-86.9023,4041,114,NaN,NaN,...,82.415817,NaN,493.0,NaN,84000001.0,USA,1377.818704,12.471541,67557.0,2.821084
36904,36904,Alabama,US,2020-04-15 22:56:51,32.3182,-86.9023,4307,118,NaN,NaN,...,87.840863,NaN,525.0,NaN,84000001.0,USA,1444.428468,12.883436,70823.0,2.739726
27038,27038,Alabama,US,2020-04-16 23:30:51,32.3182,-86.9023,4465,133,NaN,NaN,...,91.063258,NaN,553.0,NaN,84000001.0,USA,1506.734908,12.727273,73878.0,2.978723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25065,25065,Wyoming,US,2022-03-22 04:31:02,42.7560,-107.3025,155907,1769,NaN,NaN,...,26938.155605,1383108.0,NaN,1.134651,84000056.0,USA,238978.227552,NaN,NaN,NaN
8817,8817,Wyoming,US,2022-03-23 04:31:22,42.7560,-107.3025,155988,1783,NaN,NaN,...,26952.151068,1383108.0,NaN,1.143037,84000056.0,USA,238978.227552,NaN,NaN,NaN
16476,16476,Wyoming,US,2022-03-24 04:31:09,42.7560,-107.3025,155988,1783,NaN,NaN,...,26952.151068,1385758.0,NaN,1.143037,84000056.0,USA,239436.103801,NaN,NaN,NaN
22163,22163,Wyoming,US,2022-03-25 04:31:26,42.7560,-107.3025,155988,1783,NaN,NaN,...,26952.151068,1385758.0,NaN,1.143037,84000056.0,USA,239436.103801,NaN,NaN,NaN


In [12]:
count = df['Province_State'].value_counts()
print(count)


Province_State
Alabama                 713
Alaska                  713
Arizona                 713
Arkansas                713
California              713
Colorado                713
Connecticut             713
Delaware                713
District of Columbia    713
Florida                 713
Georgia                 713
Guam                    713
Hawaii                  713
Idaho                   713
Illinois                713
Indiana                 713
Iowa                    713
Kansas                  713
Kentucky                713
Louisiana               713
Maine                   713
Maryland                713
Massachusetts           713
Michigan                713
Minnesota               713
Mississippi             713
Missouri                713
Montana                 713
Nebraska                713
Nevada                  713
New Hampshire           713
New Jersey              713
New Mexico              713
New York                713
North Carolina          713
North

In [13]:
dropcol = df.columns.difference(["Last_Update", "Incident_Rate", "Confirmed", "Deaths", "Province_State"])
df.drop(dropcol, axis=1, inplace=True)


In [14]:
df

,Province_State,Last_Update,Confirmed,Deaths,Incident_Rate
18913,Alabama,2020-04-12 23:18:15,3667,93,74.788122
23035,Alabama,2020-04-13 23:07:54,3870,99,78.928288
19901,Alabama,2020-04-14 23:33:31,4041,114,82.415817
36904,Alabama,2020-04-15 22:56:51,4307,118,87.840863
27038,Alabama,2020-04-16 23:30:51,4465,133,91.063258
...,...,...,...,...,...
25065,Wyoming,2022-03-22 04:31:02,155907,1769,26938.155605
8817,Wyoming,2022-03-23 04:31:22,155988,1783,26952.151068
16476,Wyoming,2022-03-24 04:31:09,155988,1783,26952.151068
22163,Wyoming,2022-03-25 04:31:26,155988,1783,26952.151068


In [15]:
missing_values = df.isnull().sum()
print(missing_values)


Province_State    0
Last_Update       0
Confirmed         0
Deaths            0
Incident_Rate     0
dtype: int64


###As we can see there are no missing values in the dataframe. So we don't have to fill the columns with 0.

In [16]:
df["Last_Update"] = pd.to_datetime(df["Last_Update"])

df["year"] = df["Last_Update"].dt.year
df["month"] = df["Last_Update"].dt.month
df["day of the week"] = df["Last_Update"].dt.dayofweek
df["day of the month"] = df["Last_Update"].dt.day

In [17]:
df.drop(columns=["Last_Update"], inplace=True)

In [18]:
df

,Province_State,Confirmed,Deaths,Incident_Rate,year,month,day of the week,day of the month
18913,Alabama,3667,93,74.788122,2020,4,6,12
23035,Alabama,3870,99,78.928288,2020,4,0,13
19901,Alabama,4041,114,82.415817,2020,4,1,14
36904,Alabama,4307,118,87.840863,2020,4,2,15
27038,Alabama,4465,133,91.063258,2020,4,3,16
...,...,...,...,...,...,...,...,...
25065,Wyoming,155907,1769,26938.155605,2022,3,1,22
8817,Wyoming,155988,1783,26952.151068,2022,3,2,23
16476,Wyoming,155988,1783,26952.151068,2022,3,3,24
22163,Wyoming,155988,1783,26952.151068,2022,3,4,25


###Prepare time series for each Province_state

In [19]:
states = df.Province_State.unique()

states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

###1. Remove the “Province_State” column for the data

In [20]:
data = df.Province_State.unique()

data

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [21]:
def convert_aggregated_to_MA(ds,window=7):
  ds = ds.diff()

  for i in range(0,len(ds)):
    if((ds.iloc[i]<0)):
      if(ds.iloc[i-1]>0):
        ds.iloc[i]=ds.iloc[i-1]
      else:
        ds.iloc[i]=0
  return ds.rolling(window).mean()

In [22]:
global_train = None
global_val = None
global_test = None

In [32]:
# iterate through each state in the list of unique states
for state in states:
    # get the data for the current state
    state_data = df[df['Province_State'] == state]
    # print out the state name along with the corresponding data

    state_data = state_data.drop('Province_State', axis=1)

    state_data['Confirmed'] = convert_aggregated_to_MA(state_data['Confirmed'])
    state_data['Deaths'] = convert_aggregated_to_MA(state_data['Deaths'])

    state_data = state_data.dropna()

    # Check if state_data is empty after dropping NaNs
    if state_data.empty:
        print(f"Skipping state {state} due to no data after preprocessing.")
        continue  # Skip to the next state

    state_data

    #get the number of samples
    n = len(state_data)


    # use,  the first 60% of samples for training
    num_train_samples = int(n*0.6)

    #use the next 15% of samples for vaidation
    num_val_samples=int(n*0.15)

    #use the last 25% of samples for testing
    num_test_samples = int(n*0.25)

    print("number of train samples", num_train_samples)
    print("validation shape", num_val_samples)
    print("test shape", num_test_samples)

    train_mean = state_data[:num_train_samples].mean(axis=0)
    train_std = state_data[:num_train_samples].std(axis=0)
    state_data = (state_data - train_mean) / train_std

    state_data.head()



    #sequence_length is the number of past observations you want to use for prediction.
    sequence_length = 60
    batch_size = 256

    # delay is the forecasting interval, that is, how many time steps into the future you want to predict.
    # we want to forecast the deaths 30 days into the future
    delay=sequence_length+30-1

    train_dataset = keras.preprocessing.timeseries_dataset_from_array(
    data=state_data[:-delay],
    targets=state_data['Deaths'][delay:],
    sequence_length=sequence_length,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)


    val_dataset = keras.preprocessing.timeseries_dataset_from_array(
    data=state_data[:-delay],
    targets=state_data['Deaths'][delay:],
    sequence_length=sequence_length,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)


    test_dataset = keras.preprocessing.timeseries_dataset_from_array(
    data=state_data[:-delay].to_numpy(),
    targets=state_data['Deaths'][delay:],
    sequence_length=sequence_length,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

    for samples, targets in test_dataset.take(1):
      print(f"samples.shape:{samples.shape}  targets.shape:{targets.shape}")
      print(f"x={samples[0]},y={targets[0]}")

    print(state)
    print(state_data)





    if global_train is None:
        global_train = train_dataset
    else:
        # Check if the dataset is empty before concatenating
        if not train_dataset.element_spec[0].shape[0] == 0:
            global_train = global_train.concatenate(train_dataset)

    if global_val is None:
        global_val = val_dataset
    else:
        # Check if the dataset is empty before concatenating
        if not val_dataset.element_spec[0].shape[0] == 0:
            global_val = global_val.concatenate(val_dataset)

    if global_test is None:
        global_test = test_dataset
    else:
        # Check if the dataset is empty before concatenating
        if not test_dataset.element_spec[0].shape[0] == 0:
            global_test = global_test.concatenate(test_dataset)

Streaming output truncated to the last 5000 lines.
 [ 2.72018175e+00  2.22844566e-02  2.93015992e+00  1.23665281e+00
   1.12814884e+00 -4.94265773e-01 -1.09833418e+00]
 [ 2.69784510e+00  1.75243274e+00  2.96598569e+00  1.23665281e+00
   1.12814884e+00  3.53046981e-03 -9.85101090e-01]
 [ 2.47894591e+00  1.81209303e+00  2.99491861e+00  1.23665281e+00
   1.12814884e+00  5.01326713e-01 -8.71868003e-01]
 [ 2.23145581e+00  1.93141360e+00  3.01453343e+00  1.23665281e+00
   1.12814884e+00  9.99122956e-01 -7.58634916e-01]
 [ 2.10547710e+00  2.11039446e+00  3.04006247e+00  1.23665281e+00
   1.12814884e+00  1.49691920e+00 -6.45401828e-01]
 [ 2.64155674e+00  2.11039446e+00  3.03976464e+00  1.23665281e+00
   1.12814884e+00 -1.48985826e+00 -5.32168741e-01]
 [ 2.64155674e+00  2.11039446e+00  3.03976464e+00  1.23665281e+00
   1.12814884e+00 -9.92062016e-01 -4.18935654e-01]
 [ 1.84011767e+00  1.57345189e+00  3.03972209e+00  1.23665281e+00
   1.12814884e+00 -4.94265773e-01 -3.05702567e-01]
 [ 1.88568444

In [33]:
state_data.isna().sum()

,0
Confirmed,0
Deaths,0
Incident_Rate,0
year,0
month,0
day of the week,0
day of the month,0


In [29]:
for samples, targets in global_test.take(1):
    print(f"samples.shape:{samples.shape}  targets.shape:{targets.shape}")
    print(f"x={samples[0]},y={targets[0]}")

samples.shape:(30, 60, 7)  targets.shape:(30,)
x=[[ 1.00510073  1.69682866  2.63117208  1.23665281  0.82086812  0.00353047
   1.61925992]
 [ 0.77393882  1.53095865  2.63972772  1.23665281  1.12814884  0.50132671
  -1.66449961]
 [ 0.67336116  1.3894813   2.64900218  1.23665281  1.12814884  0.99912296
  -1.55126653]
 [ 0.47095556  1.26751807  2.65644187  1.23665281  1.12814884  1.4969192
  -1.43803344]
 [ 0.36801627  1.27239659  2.66092578  1.23665281  1.12814884 -1.48985826
  -1.32480035]
 [ 0.2488234   1.26263954  2.66462552  1.23665281  1.12814884 -0.99206202
  -1.21156726]
 [ 0.18839346  0.9748063   2.67241205  1.23665281  1.12814884 -0.49426577
  -1.09833418]
 [ 0.06767248  1.09676954  2.67880616  1.23665281  1.12814884  0.00353047
  -0.98510109]
 [ 0.00654793  0.92114248  2.68514999  1.23665281  1.12814884  0.50132671
  -0.871868  ]
 [-0.10403193  0.85772159  2.69042312  1.23665281  1.12814884  0.99912296
  -0.75863492]
 [-0.18196572  0.97968483  2.69504276  1.23665281  1.12814884 

In [34]:
def evaluate_model(dataset, model=None):
  total_abs_err = 0.
  num_sequences = 0
  temp_index=1
  for samples, targets in dataset:


    if model==None:

      preds = samples[:, -1, temp_index]

    #if model is not None, get its predictions
    else:
      preds= model.predict(samples, verbose=0).flatten()



  # add the absolute difference between the predictions and taragets to total_abs_err
    total_abs_err += np.sum(np.abs(preds - targets))

    # add the number of sequences in this batch to num_sequences
    num_sequences += samples.shape[0]

  #get the average absolute error
  return total_abs_err / num_sequences


print(f"Validation MAE Baseline: {evaluate_model(global_val):.2f}")
print(f"Test MAE Baseline: {evaluate_model(global_test):.2f}")


Validation MAE Baseline: 0.96
Test MAE Baseline: 0.94


In [ ]:
#every sample in train/validation/test is a 2d tensor of shape (sequence_length, num_features)
num_features= state_data.shape[-1]
def build_fcc(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.Flatten()(inputs)
  x = layers.Dense(16, activation="relu")(x)
  outputs = layers.Dense(1)(x)
  fcc = keras.Model(inputs, outputs)
  return fcc

fully_connected_model= build_fcc(sequence_length, num_features)
fully_connected_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])


# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("temp_fully_connected.keras",save_best_only=True)

#stop trianing if validation loss does not imrove for 5 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-3, restore_best_weights=True)


history = fully_connected_model.fit(global_train,epochs=50,validation_data=global_val,callbacks=[checkpoint, early_stopping])



In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

###As we can see from the above plot our model overfits.

In [ ]:

# load the checkpointed model and compute its test MAE
fully_connected_model = keras.models.load_model("temp_fully_connected.keras")

print(f"Validation MAE fully connected: {evaluate_model(global_val, model=fully_connected_model):.2f}")


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

fully_connected_model = Sequential()
fully_connected_model.add(Dense(64, activation='relu', input_dim=100))
fully_connected_model.add(Dropout(0.5))
fully_connected_model.add(Dense(64, activation='relu'))
fully_connected_model.add(Dropout(0.5))
fully_connected_model.add(Dense(1, activation='sigmoid'))


In [ ]:
#every sample in train/validation/test is a 2d tensor of shape (sequence_length, num_features)
num_features= state_data.shape[-1]
def build_fcc(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.Flatten()(inputs)
  x = layers.Dense(16, activation="relu")(x)
  outputs = layers.Dense(1)(x)
  fcc = keras.Model(inputs, outputs)
  return fcc

fully_connected_model= build_fcc(sequence_length, num_features)
fully_connected_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])


# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("temp_fully_connected.keras",save_best_only=True)

#stop trianing if validation loss does not imrove for 5 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-3, restore_best_weights=True)


history = fully_connected_model.fit(global_train,
epochs=50,
validation_data=global_val,
callbacks=[checkpoint, early_stopping])



In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

In [ ]:

# load the checkpointed model and compute its test MAE
fully_connected_model = keras.models.load_model("temp_fully_connected.keras")

print(f"Validation MAE fully connected: {evaluate_model(global_val, model=fully_connected_model):.2f}")


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2

fully_connected_model = Sequential()
fully_connected_model.add(Dense(64, activation='relu', input_dim=100, kernel_regularizer=l2(0.01)))
fully_connected_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
fully_connected_model.add(Dense(1, activation='sigmoid'))


In [ ]:
#every sample in train/validation/test is a 2d tensor of shape (sequence_length, num_features)
num_features= state_data.shape[-1]
def build_fcc(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.Flatten()(inputs)
  x = layers.Dense(16, activation="relu")(x)
  outputs = layers.Dense(1)(x)
  fcc = keras.Model(inputs, outputs)
  return fcc

fully_connected_model= build_fcc(sequence_length, num_features)
fully_connected_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])


# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("temp_fully_connected.keras",save_best_only=True)

#stop trianing if validation loss does not imrove for 5 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-3, restore_best_weights=True)


history = fully_connected_model.fit(global_train,
epochs=50,
validation_data=global_val,
callbacks=[checkpoint, early_stopping])



In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

In [ ]:

# load the checkpointed model and compute its test MAE
fully_connected_model = keras.models.load_model("temp_fully_connected.keras")

print(f"Validation MAE fully connected: {evaluate_model(global_val, model=fully_connected_model):.2f}")


In [ ]:
!pip install -q -U keras-tuner

In [ ]:
def model_builder(hp):

  #creating a placeholder for each hyperparameter with a range of values for each hyperparameter we want to tune
  hp_units = hp.Int('units', min_value = 16, max_value = 512, step = 32)
  hp_num_layers = hp.Int('num_layers', min_value = 1, max_value = 10, step=1)
  hp_dropout_rate = hp.Float('dropout', min_value = 0,max_value = 0.5, step = 0.1)
  hp_learningrate=hp.Float("initial_learning_rate",min_value=1e-4, max_value=1e-2, sampling='log')
  hp_momentum=hp.Choice("momentum", values=(0.9, 0.99))
  hp_lr_decay_steps=hp.Choice("lr_decay_steps", values=(1000, 5000, 10000))
  hp_weight_decay=hp.Float("weight_decay",min_value=1e-3, max_value=1e-1, sampling='log')


  #creating and compiling model as usual but replace the constant values for the hyperparameters you want to tune with their placeholders defined above
  input = keras.Input(shape=(sequence_length,num_features))

  #this layer scales the image tensor by dividing each pixel by 255 to bring all pixels to [0,1] range
  #x = layers.experimental.preprocessing.Rescaling(1./255)(input)

  #The first layer ( flatten layer in this case) gets the input image flatten it and pass it to the first hidden layer
  x=layers.Flatten()(input)

  #hidden layers
  for _ in range(hp_num_layers):
    x=layers.Dense(hp_units)(x)
    x= layers.BatchNormalization()(x)
    x= layers.ReLU()(x)
    x=layers.Dropout(hp_dropout_rate)(x)

  #The final output layer has one neuron with sigmoid activation to output the probability of the target class ( cate or dog whichever is labled as one)
  output=layers.Dense(1)(x)
  #create a model and set its input and output and return it
  model = keras.Model(inputs=input, outputs=output)

  lr_schedule = keras.optimizers.schedules.CosineDecay(
  initial_learning_rate=hp_learningrate,
  decay_steps=hp_lr_decay_steps, alpha=1e-2)

  #configuring and compiling the model
  opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=hp_momentum, nesterov=True)
  model.compile(loss='mse', metrics=['mae'], optimizer=opt)

  return model


In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss',
                     max_epochs = 20,
                     factor = 3,
                     directory = 'hyperparameter_runs',
                     project_name = 'time-fullyconnected')

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-4, restore_best_weights=True)
tensorboard = keras.callbacks.TensorBoard(
log_dir="/content/drive/MyDrive/tensorboard_logs/hyperband",
histogram_freq=1
)
#remove the log directory if already exists
!rm -rf "/content/drive/MyDrive/tensorboard_logs/hyperband"
tuner.search(global_train,
            validation_data = (global_val),
            epochs = 30,
            callbacks=[early_stopping, tensorboard],
            verbose = 1)

In [ ]:
#every sample in train/validation/test is a 2d tensor of shape (sequence_length, num_features)
num_features= state_data.shape[-1]
def build_fcc(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.Flatten()(inputs)
  x = layers.Dense(16, activation="relu")(x)
  outputs = layers.Dense(1)(x)
  fcc = keras.Model(inputs, outputs)
  return fcc

model= build_fcc(sequence_length, num_features)
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])


# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("temp_fully_connected.keras",save_best_only=True)

#stop trianing if validation loss does not imrove for 5 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-3, restore_best_weights=True)


history = model.fit(global_train,
epochs=50,
validation_data=global_val,
callbacks=[checkpoint, early_stopping])



In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

In [ ]:

# load the checkpointed model and compute its test MAE
model = keras.models.load_model("temp_fully_connected.keras")

print(f"Validation MAE fully connected: {evaluate_model(global_val, model=model):.2f}")


In [ ]:
#every sample in train/validation/test is a 2d tensor of shape (sequence_length, num_features)

def build_lstm(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.LSTM(16)(inputs)
  outputs = layers.Dense(1)(x)
  lstm_model = keras.Model(inputs, outputs)
  return lstm_model


lstm_model= build_lstm(sequence_length, num_features)

# check point the model with lowest validation loss
checkpoint= keras.callbacks.ModelCheckpoint("temp_lstm.keras",save_best_only=True)

#stop trianing if validation loss does not imrove for 5 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-4, restore_best_weights=True)


lstm_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = lstm_model.fit(global_train,
epochs=50,
validation_data=global_val,
callbacks=[checkpoint, early_stopping])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

In [ ]:

# load the checkpointed model and compute its test MAE
model = keras.models.load_model("temp_lstm.keras")

print(f"Validation MAE fully connected: {evaluate_model(global_val, model=lstm_model):.2f}")


### regularization

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

lstm_model = Sequential()
lstm_model.add(Dense(64, activation='relu', input_dim=100))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation='sigmoid'))


In [ ]:
#every sample in train/validation/test is a 2d tensor of shape (sequence_length, num_features)
num_features= state_data.shape[-1]
def build_fcc(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.Flatten()(inputs)
  x = layers.Dense(32, activation="relu")(x)
  outputs = layers.Dense(1)(x)
  fcc = keras.Model(inputs, outputs)
  return fcc

lstm_model= build_fcc(sequence_length, num_features)
lstm_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])


# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("regularly_connected.keras",save_best_only=True)

#stop trianing if validation loss does not imrove for 5 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-3, restore_best_weights=True)


history = lstm_model.fit(global_train,
epochs=50,
validation_data=global_val,
callbacks=[checkpoint, early_stopping])



In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

In [ ]:

# load the checkpointed model and compute its test MAE
lstm_model = keras.models.load_model("regularly_connected.keras")

print(f"Validation MAE Recuurent: {evaluate_model(global_val, model=lstm_model):.2f}")


In [ ]:
def model_builder(hp):

  #creating a placeholder for each hyperparameter with a range of values for each hyperparameter we want to tune
  hp_units = hp.Int('units', min_value = 16, max_value = 512, step = 32)
  hp_num_layers = hp.Int('num_layers', min_value = 1, max_value = 10, step=1)
  hp_dropout_rate = hp.Float('dropout', min_value = 0,max_value = 0.5, step = 0.1)
  hp_learningrate=hp.Float("initial_learning_rate",min_value=1e-4, max_value=1e-2, sampling='log')
  hp_momentum=hp.Choice("momentum", values=(0.9, 0.99))
  hp_lr_decay_steps=hp.Choice("lr_decay_steps", values=(1000, 5000, 10000))
  hp_weight_decay=hp.Float("weight_decay",min_value=1e-3, max_value=1e-1, sampling='log')


  #creating and compiling model as usual but replace the constant values for the hyperparameters you want to tune with their placeholders defined above
  input = keras.Input(shape=(sequence_length,num_features))

  #this layer scales the image tensor by dividing each pixel by 255 to bring all pixels to [0,1] range
  #x = layers.experimental.preprocessing.Rescaling(1./255)(input)

  #The first layer ( flatten layer in this case) gets the input image flatten it and pass it to the first hidden layer
  x=layers.Flatten()(input)

  #hidden layers
  for _ in range(hp_num_layers):
    x=layers.Dense(hp_units)(x)
    x= layers.BatchNormalization()(x)
    x= layers.ReLU()(x)
    x=layers.Dropout(hp_dropout_rate)(x)

  #The final output layer has one neuron with sigmoid activation to output the probability of the target class ( cate or dog whichever is labled as one)
  output=layers.Dense(1)(x)
  #create a model and set its input and output and return it
  model_2 = keras.Model(inputs=input, outputs=output)

  lr_schedule = keras.optimizers.schedules.CosineDecay(
  initial_learning_rate=hp_learningrate,
  decay_steps=hp_lr_decay_steps, alpha=1e-2)

  #configuring and compiling the model
  opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=hp_momentum, nesterov=True)
  model_2.compile(loss='mse', metrics=['mae'], optimizer=opt)

  return model_2


In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss',
                     max_epochs = 20,
                     factor = 3,
                     directory = 'hyperparameter_runs',
                     project_name = 'time-recuurent-fullyconnected')

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-4, restore_best_weights=True)
tensorboard = keras.callbacks.TensorBoard(
log_dir="/content/drive/MyDrive/tensorboard_logs/hyperband_1",
histogram_freq=1
)
#remove the log directory if already exists
!rm -rf "/content/drive/MyDrive/tensorboard_logs/hyperband_1"
tuner.search(global_train,
            validation_data = (global_val),
            epochs = 30,
            callbacks=[early_stopping, tensorboard],
            verbose = 1)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss recurrent baseline')
plt.legend()
plt.show()

In [ ]:

# load the checkpointed model and compute its test MAE
model_2 = keras.models.load_model("temp_lstm.keras")

print(f"Validation MAE fully connected: {evaluate_model(global_val, model=model_2):.2f}")


In [ ]:
def build_stacked_lstm(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))
  x = layers.GRU(32, recurrent_dropout=0.5,return_sequences=True)(inputs)
  x = layers.GRU(32, recurrent_dropout=0.5)(x)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1)(x)
  model = keras.Model(inputs, outputs)
  return model

stacked_lstm_model= build_stacked_lstm(sequence_length, num_features)
# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("temp_stacked_lstm.keras",save_best_only=True)

stacked_lstm_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = stacked_lstm_model.fit(train_dataset,
epochs=50,
validation_data=val_dataset,
callbacks=[checkpoint, early_stopping])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# load the checkpointed model and compute its test MAE
stacked_lstm_model = keras.models.load_model("temp_stacked_lstm.keras")

print(f"Validation MAE lstm: {evaluate_model(val_dataset, model=stacked_lstm_model):.2f}")


In [ ]:
def build_1dconv(sequence_length, num_features):
  inputs = keras.Input(shape=(sequence_length, num_features))

  # a 1D convolutional layer with 8 filters and a kernel/window of size 14
  x = layers.Conv1D(8, 14, activation="relu")(inputs)
  x = layers.MaxPooling1D(2)(x)

  # a 1D convolutional layer with 8 filters and a kernel of size 14
  x = layers.Conv1D(8, 14, activation="relu")(x)
  x = layers.MaxPooling1D(2)(x)


  # a golobal average pooling layer that takes the global average along the temporal dimension
  x = layers.GlobalAveragePooling1D()(x)
  outputs = layers.Dense(1)(x)
  model = keras.Model(inputs, outputs)
  return model
conv1D_model= build_1dconv(sequence_length, num_features)

conv1D_model.summary()



In [ ]:
# check point the best model
checkpoint= keras.callbacks.ModelCheckpoint("temp_conv1d.keras",save_best_only=True)

conv1D_model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = conv1D_model.fit(train_dataset,
epochs=50,
validation_data=val_dataset,
callbacks=[checkpoint, early_stopping])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# load the checkpointed model and compute its test MAE
conv1D_model = keras.models.load_model("temp_conv1d.keras")

print(f"Validation MAE lstm: {evaluate_model(val_dataset, model=conv1D_model):.2f}")

In [ ]:
# load the checkpointed model and compute its test MAE
best_model=keras.models.load_model("temp_stacked_lstm.keras")

print(f"test MAE for best model: {evaluate_model(test_dataset, model=best_model):.2f}")
print(f"test MAE for commonsense model: {evaluate_model(global_test):.2f}")